Для теста используется датасет MTS ML Cup – 322млн.899435 строк – история активности пользователей в интернете

[Скачать архив с датасетом](https://storage.yandexcloud.net/ds-ods/files/materials/124f46f0/competition_data_final_pqt.zip)



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
import pandas as pd
import polars as pl
import dask.dataframe as dd

In [2]:
DATASET = '../mts/dataset/competition_data_final_pqt'

# PYSPARK

In [3]:
%%time
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('myAppName') \
    .config('spark.driver.memory','4g') \
    .config('spark.ui.port',4041) \
    .getOrCreate()
spark.sparkContext.setLogLevel('FATAL')

df = spark.read.parquet(DATASET)

df.select(split(df.url_host,'\.').alias('url_parts')).show(20,False)
df.groupBy('cpe_model_os_type').count().show()

print('\nTotal rows',df.count())
del(df)
spark.stop()

23/07/13 21:16:58 WARN Utils: Your hostname, MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.162 instead (on interface en0)
23/07/13 21:16:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/13 21:16:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------------------+
|url_parts                       |
+--------------------------------+
|[apple, com]                    |
|[apple, com]                    |
|[banners, adfox, ru]            |
|[apple, com]                    |
|[instagram, com]                |
|[yastatic, net]                 |
|[apple, com]                    |
|[googleads, g, doubleclick, net]|
|[relap, io]                     |
|[yastatic, net]                 |
|[zen, yandex, ru]               |
|[apple, com]                    |
|[avatars, mds, yandex, net]     |
|[instagram, com]                |
|[interesbook, com]              |
|[ok, ru]                        |
|[online, sberbank, ru]          |
|[online, sberbank, ru]          |
|[vk, com]                       |
|[yandex, ru]                    |
+--------------------------------+
only showing top 20 rows



+-----------------+---------+
|cpe_model_os_type|    count|
+-----------------+---------+
|              iOS|122764510|
|        Apple iOS|  2275740|
|          Android|197859185|
+-----------------+---------+


Total rows 322899435
CPU times: user 21.1 ms, sys: 14.7 ms, total: 35.9 ms
Wall time: 20.2 s


# POLARS

In [4]:
%%time
df = pl.scan_parquet(f'{DATASET}/*.parquet')
print(df.select(pl.col('url_host').str.split(by='.')).head(20).collect())
print(df.groupby('cpe_model_os_type').count().collect())
del(df)

shape: (20, 1)
┌───────────────────────────────────┐
│ url_host                          │
│ ---                               │
│ list[str]                         │
╞═══════════════════════════════════╡
│ ["ad", "adriver", "ru"]           │
│ ["apple", "com"]                  │
│ ["avatars", "mds", … "net"]       │
│ ["googleads", "g", … "net"]       │
│ …                                 │
│ ["googleads", "g", … "net"]       │
│ ["icloud", "com"]                 │
│ ["mail", "yandex", "ru"]          │
│ ["syndication", "realsrv", "com"… │
└───────────────────────────────────┘


PARTITIONED DS


shape: (3, 2)
┌───────────────────┬───────────┐
│ cpe_model_os_type ┆ count     │
│ ---               ┆ ---       │
│ str               ┆ u32       │
╞═══════════════════╪═══════════╡
│ Android           ┆ 197859185 │
│ iOS               ┆ 122764510 │
│ Apple iOS         ┆ 2275740   │
└───────────────────┴───────────┘
CPU times: user 10.7 s, sys: 1.06 s, total: 11.8 s
Wall time: 3.87 s


# DASK

In [5]:
%%time
df = dd.read_parquet(DATASET)
print(df.url_host.str.split('\.').head(20))
# print(df.groupby('cpe_model_os_type').cpe_model_os_type.count().compute())
del(df)

0                    [ad, adriver, ru]
1                         [apple, com]
2          [avatars, mds, yandex, net]
3     [googleads, g, doubleclick, net]
4     [googleads, g, doubleclick, net]
5                      [i, ytimg, com]
6                        [icloud, com]
7                       [m, avito, ru]
8                          [relap, io]
9               [sun9-5, userapi, com]
10                    [ads, adfox, ru]
11                        [apple, com]
12                        [apple, com]
13                [banners, adfox, ru]
14    [googleads, g, doubleclick, net]
15    [googleads, g, doubleclick, net]
16    [googleads, g, doubleclick, net]
17                       [icloud, com]
18                  [mail, yandex, ru]
19         [syndication, realsrv, com]
Name: url_host, dtype: object
CPU times: user 20.8 s, sys: 1.32 s, total: 22.1 s
Wall time: 22.4 s


# PANDAS

In [6]:
# %%time
# df = pd.read_parquet(DATASET,columns=['url_host','cpe_model_os_type'])
# print(df.url_host.str.split('\.').head(20))
# print(df.groupby('cpe_model_os_type').cpe_model_os_type.count())
# del(df)